In [1]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

# Ejectuamos la búsqueda
years = [2019, 2020, 2021, 2022, 2023] 
data = nfl.import_pbp_data(years = years)


2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [2]:
# Seleccionamos las observaciones pertinentes al proyecto
fourth_down_data_row = data[(data['down'] == 4) & 
                            (data['play_type'].isin(['pass', 'run'])) &
                            (data['replay_or_challenge'] == 0) &
                            (data['penalty'] == 0)] 

# Definimos las variables de interés
scoped_variables = ['yardline_100', 'posteam_type', 'half_seconds_remaining', 'game_half', 'goal_to_go', 'ydstogo', 'ydsnet', 'play_type',
                    'shotgun', 'no_huddle', 'qb_dropback', 'pass_length', 'pass_location', 'run_location', 'run_gap', 'score_differential',
                    'fourth_down_converted']

# Nos quedamos con el scope final (row data)
fourth_down_data = fourth_down_data_row[scoped_variables]

# Combinamos las columnas de run & pass location (porque son los mismo valores únicos)
fourth_down_data['play_location'] = np.where(fourth_down_data['play_type'] == 'pass', fourth_down_data['pass_location'], fourth_down_data['run_location'])
fourth_down_data = fourth_down_data.dropna(subset=['play_location']) 

# Quitamos las variables de pass & run locations
fourth_down_data = fourth_down_data.drop(columns = ['pass_location', 'run_location'])

# Combinamos las columnas de run & pass location (porque son los mismos valores únicos)
fourth_down_data['play_subtype'] = np.where(fourth_down_data['play_type'] == 'pass', 
                                             ('Pass: ' + fourth_down_data['pass_length']), ('Run: ' + fourth_down_data['run_gap']))

fourth_down_data = fourth_down_data.dropna(subset=['play_subtype']) 

# Quitamos las variables de pass & run locations
fourth_down_data = fourth_down_data.drop(columns = ['pass_length', 'run_gap'])

# Generamos una nueva columna con rangos para la variable yardline_100
bins = list(range(0, 101, 10))
labels = [f"{i}-{i+9}" for i in bins[:-1]]
fourth_down_data['yardline_group'] = pd.cut(fourth_down_data['yardline_100'], bins=bins, labels=labels, right=False)

# Asignamos etiquetas a la variable y
fourth_down_data['fourth_down_converted'] = fourth_down_data['fourth_down_converted'].replace({0: 'Not converted', 1: 'Converted'})

# Guardamos la data final como un csv para facilitar tanto el modelo como el EDA
fourth_down_data.to_csv("C:/Users/rafae/OneDrive/Documentos/Personal/Ciencia de datos/TOG/Final_docs/fourth_down_data.csv", index=False)

C:\Users\rafae\AppData\Local\Temp\ipykernel_1464\4212933451.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down_data['play_location'] = np.where(fourth_down_data['play_type'] == 'pass', fourth_down_data['pass_location'], fourth_down_data['run_location'])


In [ ]:
fourth_down_data.head(50)